Façamos uma crifra simples que só permite letras do alfabeto, tudo será convertido em maiúsculo e todos símbolos extra serão desconsiderados. A chave da mesma forma deve possuir apenas letras do alfabeto sendo 26 o número de possibilidades de cada símbolo.

In [189]:
def validate_key(secret: str) -> bool:
    return secret.isalpha()

def cypher(secret: str, message: str):
    if not validate_key(secret):
        raise ValueError("Secret must consist of letters only.")

    secret = secret.upper()

    result, secret_position = [], 0
    for symbol in message:
        if symbol.isalpha():
            secret_index = secret_position % len(secret)
            secret_shift = ord(secret[secret_index]) - ord("A")

            base = ord('A') if symbol.isupper() else ord('a')
            symbol_ord = ord(symbol.upper()) - ord("A")
            encrypted_symbol = chr(base + (symbol_ord + secret_shift) % 26)

            secret_position += 1

            result.append(encrypted_symbol)
        else:
            result.append(symbol)

    return "".join(result)


In [32]:
cypher("LEMON", "Attack at dawn.")

'Lxfopv ef rnhr.'

In [190]:
def decypher(secret: str, message: str):
    if not validate_key(secret):
        raise ValueError("Secret must consist of letters only.")

    secret = secret.upper()

    result, secret_position = [], 0
    for symbol in message:
        if symbol.isalpha():
            secret_index = secret_position % len(secret)
            secret_shift = ord(secret[secret_index]) - ord("A")

            base = ord('A') if symbol.isupper() else ord('a')
            symbol_ord = ord(symbol.upper()) - ord("A")
            encrypted_symbol = chr(base + abs((symbol_ord - secret_shift) % 26))

            secret_position += 1

            result.append(encrypted_symbol)
        else:
            result.append(symbol)

    return "".join(result)

In [191]:
decypher("LEMON", "Lxfopv ef rnhr.")

'Attack at dawn.'

# Começando o ataque

Vamos agora tentar atacar esse método. Primeiro apresentamos um texto maior que possiblita a análise de frequência. A mensagem é a seguinte:

"Throughout history, the art of secret writing has evolved from simple substitution ciphers to the most advanced public key cryptosystems. Early methods relied heavily on letter frequency patterns, making them vulnerable to analytical attacks. Modern cryptography blends mathematics, computer science, and information theory to ensure confidentiality and integrity in a digital world. Researchers constantly probe algorithms to identify weaknesses and strengthen protocols. By experimenting with challenging but known plaintexts, learners develop intuition about how cryptanalysis works in practice. This extended sample contains diverse punctuation marks, spaces, and varying word lengths to simulate a realistic message that still allows for systematic analysis. Users can encrypt this passage using a repeating Vigenere key and then apply their attack strategies to recover the key length and internal structure based on statistical methods."

Perceba que, da forma como foi implementado, o nosso cifrador irá ignorar espaços e pontuação, esse serão preservados na mensagem da forma como estão. No entanto, acentos ortográficos seriam desconsiderados, então Vigenère seria criptografado apenas a parte Vigenre e o è permaneceria como está. No texto acima evitamos essa peculiaridade propositadamente.

Obtemos como resultado o seguinte texto cifrado:

In [192]:
key = "CRYPTOGRAPHY"
message = "Throughout history, the art of secret writing has evolved from simple substitution ciphers to the most advanced public key cryptosystems. Early methods relied heavily on letter frequency patterns, making them vulnerable to analytical attacks. Modern cryptography blends mathematics, computer science, and information theory to ensure confidentiality and integrity in a digital world. Researchers constantly probe algorithms to identify weaknesses and strengthen protocols. By experimenting with challenging but known plaintexts, learners develop intuition about how cryptanalysis works in practice. This extended sample contains diverse punctuation marks, spaces, and varying word lengths to simulate a realistic message that still allows for systematic analysis. Users can encrypt this passage using a repeating Vigenere key and then apply their attack strategies to recover the key length and internal structure based on statistical methods."

encrypted = cypher(key, message)
encrypted

'Vypdnunfui ogukmgr, hnv aga mh jcrksz nrxagpx fpl sbflklb himb lwsglt zsdjrxmizzoc jgrycgl hu kht tmuk ysoottes wsdcgr dse trnwrqjwhmssj. Epyja dciacjj rtsggu fttjocy du jgkrtk txvqjllep npmhkinh, tymzlv mvkd vjslgiyqes zf achjakgrtz gktpjiu. Dmsxft trnwrqxppive sltubu dyiassrtxjq, efkenhki srpcptc, pgr oefdykckgdg hnvogf rq vlhnfk tocmgfvliborztn hlf zlixuxztn pl c ugvbhgc wdyjf. Ichxoxthtyq eflhmotkln wpqsc peuuiiioku km xwstkiuf ugricxgyvs pub ukptguzyec wpqkmrhzy. Sy tengigbxbzznv dgvy awtzrvnvpli ssi dbunn esykertqhy, cepylgiq sxjkcoe plvlgibct rbdbr jfu rkmvkachjajgh pcxbs xu ntraibqk. Khxz czkccwsj jabwjg tmcmooes sptgiqt iitttjhrkfl btfqj, sehagj, ycw jgiyxue yfps estxtwz rq jgbnzgke p ycccghmwi dehzyiv rwth ykias yncmll tui snzrgdyibq geaafqkj. Shxfy tac lleiwem hnzs ehquret ngoeg p ycrvyibbm Mivllgic zxm ged iocp rneem zyexy yvkyrd gziailekvq ih fktoklp vyc zxm rvnvaf ceb xghkinps qvisrmixv bpzcf fl hmozzsipacc ktmvuus.'

# Descobrindo o tamanho da chave
Agora vamos descobrir o tamanho da chave. Podemos fazer isso analizando combinações de duas, três ou quatro letras, depois verificamos o espaçamento entre essas combinações e computamos os fatores primos desse espaçamento e verificamos aqueles que mais se repetem. Esse é o chamado método de Kasiski.

In [46]:
def strip_nonalpha(message: str):
    result = []
    for symbol in message:
        if symbol.isalpha():
            result.append(symbol)
    
    return "".join(result)

In [54]:
def factors_up_to(n: int, max_factor: int) -> list[int]:
    if n < 2 or max_factor < 2:
        return []
    
    factors = []
    # Only need to loop up to min(sqrt(n), max_factor)
    upper = min(int(n ** 0.5), max_factor)
    for i in range(2, upper + 1):
        if n % i == 0:
            factors.append(i)
            complement = n // i
            # Add the complementary factor if within limit and not duplicate
            if complement != i and complement <= max_factor:
                factors.append(complement)
    
    # If n itself is <= max_factor, include it
    if n <= max_factor:
        factors.append(n)
    
    return sorted(factors)


In [64]:
from collections import defaultdict

def letters_frequency(encrypted: str, seq_len: int = 3, max_key_size: int = 20):
    positions: defaultdict[str, list[int]] = defaultdict(list)
    for i in range(len(encrypted) - seq_len - 1):
        combination = encrypted[i:i+seq_len]
        positions[combination].append(i)

    factors_frequency: dict[int, int] = {}
    for locs in positions.values():
        if len(locs) > 1:
            for j in range(len(locs)-1):
                spacing = locs[j+1] - locs[j]
                factors = factors_up_to(spacing, max_key_size)
                for factor in factors:
                    if factor not in factors_frequency:
                        factors_frequency[factor] = 1
                    else:
                        factors_frequency[factor] += 1

    return factors_frequency

In [65]:
plain = strip_nonalpha(encrypted).upper()
letters_frequency(plain)

{2: 48,
 3: 42,
 4: 42,
 6: 41,
 12: 38,
 7: 8,
 9: 13,
 14: 8,
 18: 13,
 5: 7,
 8: 15,
 10: 6,
 15: 7,
 16: 8,
 20: 5,
 13: 2,
 19: 5,
 11: 3,
 17: 1}

Como podemos ver os números que mais se repetem são o 2, 3, 4, 6 e 12 disparadamente. Se verificásssemos usando combinações de 2 ou 4 letras, obteríamos o mesmo resultado. Isso faz sentido já que o tamanho da nossa chave é 12 e todos os números 2, 3, 4 e 6 são fatores de 12. Na prática poderíamos tentar todas as alternativas ou observar esse fênomeno vendo a discrepância entre o 12 e os demais números (7, 9, 14, etc) e observando que todos os outros números que aparecem com frequência (2, 3, 4, 6) são fatores de 12. Portanto, chegamos no tamanho da nossa chave: 12.

# Descobrindo a chave

Agora sabemos o tamanho da chave (12), podemos então verificar cada letra da chave individualmente. Para cada letra, verificamos a mensagem nos índices 0, 12, 24, etc. Depois 1, 13, 25, etc, e assim por diante. Em cada caso comparamos as frequências dos símbolos com a frequência das letras da língua inglesa e verificamos qual dos deslocamentos mais corresponde. E fazemos assim letra por letra.

In [138]:
def frequency(encrypted: str, key_size: int, key_index: int, test_key: str):
    encrypted = strip_nonalpha(encrypted).upper()
    current_frequency: dict[str, int] = {}
    total = 0

    for i in range(key_index, len(encrypted), key_size):
        encrypted_symbol = encrypted[i]
        test_decrypted_symbol = decypher(test_key, encrypted_symbol)
        if test_decrypted_symbol not in current_frequency:
            current_frequency[test_decrypted_symbol] = 1
        else:
            current_frequency[test_decrypted_symbol] += 1

        total += 1

    result: dict[str, float] = {key: value / total for key, value in current_frequency.items()}
    return result

Vamos verificar se uma frequência é "boa" de acordo com a ordem da língua inglesa através de uma comparação estatística entre a frequência das letras da língua inglesa e a frequência das letras advinhadas, aquela que tiver uma pontuação menor de acordo com o teste "chi square" é aquela que tem a frequência mais parecida e será a nossa chave.

In [139]:
def chi_square_statistic(observed: dict[str, float], expected: dict[str, float]) -> float:
    chi2 = 0.0
    for category, E in expected.items():
        O = observed.get(category, 0.0)
        if E > 0:
            chi2 += (O - E) ** 2 / E
            
    return chi2

In [165]:
english_frequency = {
    "A": 0.08167, "B": 0.01492, "C": 0.02782, "D": 0.04253, "E": 0.12702,
    "F": 0.02228, "G": 0.02015, "H": 0.06094, "I": 0.06966, "J": 0.00153,
    "K": 0.00772, "L": 0.04025, "M": 0.02406, "N": 0.06749, "O": 0.07507,
    "P": 0.01929, "Q": 0.00095, "R": 0.05987, "S": 0.06327, "T": 0.09056,
    "U": 0.02758, "V": 0.00978, "W": 0.02360, "X": 0.00150, "Y": 0.01974,
    "Z": 0.00074,
}

In [166]:
import math

def best_key_for_index(encrypted: str, key_size: int, key_index: int):
    best_key = None
    best_score = math.inf
    for test_key in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        score = chi_square_statistic(
            frequency(encrypted, key_size, key_index, test_key), english_frequency
        )
        if score < best_score:
            best_key = test_key
            best_score = score

    return best_key

Vamos testar o nosso método com a primeira letra da chave. Como podemos ver, ele corretamente previu a letra "C" como a primeira.

In [167]:
best_key_for_index(encrypted, 12, 0)

'C'

In [168]:
def attack_cypher(encrypted: str, key_size: int):
    result = []
    for i in range(key_size):
        result.append(best_key_for_index(encrypted, key_size, i))

    return "".join(result)

E como podemos ver podemos iterar todos os indíces descobrindo a chave.

In [193]:
attack_cypher(encrypted, 12)

'CRYPTOGRAPHY'

Se estivéssemos entre os tamanhos 2, 3, 4, 6 e 12, poderíamos também compará-los usando a análise de frequência.

In [185]:
import string, math

def best_size(encrypted: str, candidates: list[int]) -> int:
    best_candidate = None
    best_score = math.inf

    for key_size in candidates:
        total_chi = 0.0
        for i in range(key_size):
            # find the best shift at this position
            min_chi = math.inf
            for test_key in string.ascii_uppercase:
                obs = frequency(encrypted, key_size, i, test_key)
                chi = chi_square_statistic(obs, english_frequency)
                if chi < min_chi:
                    min_chi = chi
            total_chi += min_chi

        avg_chi = total_chi / key_size
        if avg_chi < best_score:
            best_score = avg_chi
            best_candidate = key_size

    return best_candidate


In [174]:
best_size(encrypted, [12, 6, 4, 3, 2])

12

Farei uma função agora que irá pegar os melhores candidatos baseado na análise do Kasiski, escolhendo aqueles candidatos depois de uma variação brusca. Por exemplo, perceba que no nosso exemplo anterior, depois do 12 com 38 repetições o segundo mais frequente era o 8 com 15. Essa é uma variação bastante brusca e possivelmente uma boa heurística para selecionar os melhores candidatos.

In [175]:
def select_candidates(encrypted: str, seq_len: int = 3, max_key_size: int = 20):
    text = strip_nonalpha(encrypted).upper()
    freqs = letters_frequency(text, seq_len, max_key_size)
    items = sorted(freqs.items(), key=lambda kv: kv[1], reverse=True)
    if len(items) < 2:
        return [items[0][0]] if items else []

    # compute consecutive drops
    drops = [items[i][1] - items[i+1][1] for i in range(len(items)-1)]
    maximum_difference = drops.index(max(drops))

    return [size for size, _ in items[: maximum_difference+1]]

In [176]:
select_candidates(encrypted)

[2, 3, 4, 6, 12]

Finalmente criemos uma função que une todas as técnicas para descobrir uma senha qualquer.

In [177]:
def attack(encrypted: str):
    candidates = select_candidates(encrypted)
    size = best_size(encrypted, candidates)
    return attack_cypher(encrypted, size)

In [194]:
attack(encrypted)

'CRYPTOGRAPHY'

Testemos agora com uma outra senha. DIFFERENT

In [179]:
encrypted = cypher("DIFFERENT", message)

In [180]:
attack(encrypted)

'DIFFERENT'

Finalmente, testemos com outra mensagem. É importante que ela seja em inglês e que contenha um número de palavras suficientemente grande para que a nossa análise estatística seja bem-sucedida.

In [181]:
message = "Cryptography is the science of securing communication by converting data into unreadable formats, ensuring confidentiality. Modern systems use symmetric ciphers like AES, asymmetric methods such as RSA or elliptic curve cryptography, and hashing functions like SHA. Techniques such as digital signatures verify identity and protect information in transit and at rest from unauthorized access and tampering."
secret = "PASSWORD"
encrypted = cypher(secret, message)

In [186]:
select_candidates(encrypted)

[2, 4, 3, 6, 12, 8]

In [187]:
attack(encrypted)

'PASSWORD'